In [448]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
import keras
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from tensorflow.keras.layers import Dense, Input, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization,Add,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import LeakyReLU, ReLU, Conv2D, MaxPooling2D, BatchNormalization, Conv2DTranspose, UpSampling2D, concatenate
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras import applications, models, layers,optimizers
import glob

In [449]:
train_non = glob.glob('train/Nonparallel/*')
test_para = glob.glob('test/Parallel/*')
test_non = glob.glob('test/Nonparallel/*')
test_para = glob.glob('test/Parallel/*')

In [450]:
tr_para = pd.Series(train_para)
tr_non = pd.Series(train_non)
te_para = pd.Series(test_para)
te_non = pd.Series(test_non)

In [451]:
tr_para_df = pd.DataFrame(tr_para, columns = ['image'])
tr_para_df['Orientation'] = 1
tr_non_df = pd.DataFrame(tr_non, columns = ['image'])
tr_non_df['Orientation'] = 0
te_para_df = pd.DataFrame(te_para, columns = ['image'])
te_para_df['Orientation'] = 1
te_non_df = pd.DataFrame(te_non, columns = ['image'])
te_non_df['Orientation'] = 0

In [452]:
train = pd.concat([tr_para_df, tr_non_df])
test= pd.concat([te_para_df, te_non_df])

In [453]:
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [454]:
total_df = pd.concat([train, test])

In [455]:
total_df = total_df.reset_index(drop=True)

In [456]:
Input_shape = (299,299,3)

In [457]:
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = 22)
for train_index, test_index in split.split(total_df, total_df['Orientation']):
    train_df = total_df.loc[train_index]
    val_df = total_df.loc[test_index]

In [458]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [459]:
train_X = train_df['image']
train_Y = train_df['Orientation']
val_X = val_df['image']
val_Y = val_df['Orientation']

In [461]:
a = []
for i in range(0,len(train_X)):
    s = cv2.imread(train_X[i],cv2.IMREAD_GRAYSCALE)
    a.append(s)

In [462]:
b = []
for i in range(0,len(val_X)):
    d = cv2.imread(val_X[i],cv2.IMREAD_GRAYSCALE)
    b.append(d)

In [463]:
train_X_me = []
for i in range(0,len(a)):
    q = cv2.medianBlur(a[i], 3)
    w = cv2.resize(q, (299,299), cv2.INTER_AREA)
    e = cv2.cvtColor(w, cv2.COLOR_GRAY2BGR)
    train_X_me.append(e)

In [464]:
val_X_me = []
for i in range(0,len(b)):
    z = cv2.medianBlur(b[i], 3)
    x = cv2.resize(z, (299,299), cv2.INTER_AREA)
    c = cv2.cvtColor(x, cv2.COLOR_GRAY2BGR)
    val_X_me.append(c)

In [465]:
train_X = train_X_me
val_X = val_X_me

In [466]:
train_X = np.array(train_X)
val_X = np.array(val_X)
train_Y = np.array(train_Y)
val_Y = np.array(val_Y)

for i in range(0, len(train_X)):
    train_X[i] = cv2.resize(train_X[i], (299,299), cv2.INTER_AREA)

for i in range(0, len(val_X)):
    val_X[i] = cv2.resize(val_X[i], (299,299), cv2.INTER_AREA)

def UNet(pretrained_weights = None,input_size = (128,128,1)):
    inp = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inp)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = inp, outputs=[conv10])

    return model

model = UNet()
model.summary()
model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [483]:
IRES_Net = tf.keras.applications.InceptionResNetV2(
    include_top = False,
    weights = 'imagenet',
    input_shape = Input_shape,
)
base = IRES_Net
transfer = base.output
transfer = MaxPooling2D(pool_size = (4,4))(transfer)
transfer = Flatten()(transfer)
transfer = Dense(64, activation = 'relu')(transfer)
transfer = Dropout(0.2)(transfer)
transfer = Dense(1, activation = 'sigmoid')(transfer)

model = Model(inputs=base.input, outputs = transfer)

In [484]:
model.compile(
        optimizer=optimizers.RMSprop(learning_rate = 0.000003),
        loss="binary_crossentropy",
        metrics=["binary_accuracy"],
    )

In [490]:
model_name = "orientation"
model_weight = model_name + "_weight.h5"
model.save(model_name+".h5")

In [470]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_12 (InputLayer)          [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_2233 (Conv2D)           (None, 149, 149, 32  864         ['input_12[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_2233 (Batc  (None, 149, 149, 32  96         ['conv2d_2233[0][0]']            
 hNormalization)                )                                                           

In [325]:
model.input

<KerasTensor: shape=(None, 299, 299, 3) dtype=float32 (created by layer 'input_9')>

In [486]:
Epoch = 15

CALLBACK = [
    tf.keras.callbacks.ModelCheckpoint(filepath='.h5',save_best_only=True),
    tf.keras.callbacks.EarlyStopping(patience=3)
]


history = model.fit(
        train_X,train_Y,
        epochs=Epoch,
        callbacks=CALLBACK,
        validation_data=(val_X,val_Y),
        batch_size = 32
    )

Epoch 1/15
126/126 [==============================] - 425s 3s/step - loss: 0.5321 - binary_accuracy: 0.7834 - val_loss: 0.4533 - val_binary_accuracy: 0.8206
Epoch 2/15
126/126 [==============================] - 407s 3s/step - loss: 0.4404 - binary_accuracy: 0.8179 - val_loss: 0.4056 - val_binary_accuracy: 0.8160
Epoch 3/15
126/126 [==============================] - 404s 3s/step - loss: 0.3808 - binary_accuracy: 0.8375 - val_loss: 0.3915 - val_binary_accuracy: 0.8356
Epoch 4/15
126/126 [==============================] - 407s 3s/step - loss: 0.3511 - binary_accuracy: 0.8494 - val_loss: 0.3722 - val_binary_accuracy: 0.8391
Epoch 5/15
126/126 [==============================] - 406s 3s/step - loss: 0.3177 - binary_accuracy: 0.8687 - val_loss: 0.3563 - val_binary_accuracy: 0.8455
Epoch 6/15
126/126 [==============================] - 409s 3s/step - loss: 0.3007 - binary_accuracy: 0.8767 - val_loss: 0.3471 - val_binary_accuracy: 0.8524
Epoch 7/15
126/126 [==============================] - 404s